In [ ]:
import os
# on se replace à la racine du projet pour utiliser les modules créé dans le projet attention bien démarré le venv
os.chdir('../../../..')
from data_storage.sql_alchemy_connect.alchemy_cur import AlchemyConn
import pandas as pd



In [2]:
connector = AlchemyConn()
df = pd.read_sql('SELECT * FROM raw_data',con=connector.get_engine())

In [3]:
print(df.head())
print(df.shape)

   id  surface_reelle_bati   prix_m2  nombre_pieces_principales  longitude  \
0   3                 78.0  11643.60                        3.0    2.25112   
1   4                 70.0   8571.43                        3.0    2.25190   
2   5                 77.0  10066.60                        4.0    2.25243   
3   6                 67.0   8208.96                        3.0    2.25287   
4   7                 74.0   9331.08                        3.0    2.25137   

   latitude  nombre_lots  annee  arrondissement  distance_datashop_km  ...  \
0   48.8430            2   2020              16              1.123880  ...   
1   48.8384            4   2020              16              0.152864  ...   
2   48.8377            1   2020              16              0.769220  ...   
3   48.8374            2   2020              16              0.410568  ...   
4   48.8393            2   2020              16              0.269765  ...   

   distance_ecole_km  distance_metro_km  distance_TER_km  dist

In [4]:
y = df['prix_m2']
X = df.drop(['prix_m2'],axis=1)
print(y.head())

0    11643.60
1     8571.43
2    10066.60
3     8208.96
4     9331.08
Name: prix_m2, dtype: float64


In [5]:
print(X.isnull().sum())
print('\n-------\n')
print(y.isnull().sum())

id                           0
surface_reelle_bati          0
nombre_pieces_principales    0
longitude                    0
latitude                     0
nombre_lots                  0
annee                        0
arrondissement               0
distance_datashop_km         0
distance_espace_vert_km      0
distance_college_km          0
distance_universite_km       0
distance_ecole_km            0
distance_metro_km            0
distance_TER_km              0
distance_POI_min_km          0
proche_POI_1km               0
nb_POIs_inf_1km              0
cle_interop_adr_proche       0
distance_batiment_m          0
annee_construction_dpe       0
zone                         0
dtype: int64

-------

0


In [6]:
X.dtypes

id                             int64
surface_reelle_bati          float64
nombre_pieces_principales    float64
longitude                    float64
latitude                     float64
nombre_lots                    int64
annee                          int64
arrondissement                 int64
distance_datashop_km         float64
distance_espace_vert_km      float64
distance_college_km          float64
distance_universite_km       float64
distance_ecole_km            float64
distance_metro_km            float64
distance_TER_km              float64
distance_POI_min_km          float64
proche_POI_1km                 int64
nb_POIs_inf_1km                int64
cle_interop_adr_proche        object
distance_batiment_m          float64
annee_construction_dpe       float64
zone                           int64
dtype: object

### Les arrondissements sont difficilement OneHotEncoding

In [7]:
import plotly.express as px

X.arrondissement.value_counts()
fig = px.bar(x=X.arrondissement.unique(),y=X.arrondissement.value_counts())
fig.show()


Il serait intéressant de faire un premier clustering pour créer des catégories et faire diminuer le nombre de zones

In [8]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

cluster_df = df[['longitude','latitude']]
print(cluster_df.dtypes)
kmeans = KMeans(n_clusters=7, random_state=0)

sc = StandardScaler()
X = sc.fit_transform(cluster_df)

kmeans.fit(X)
inverse = sc.inverse_transform(kmeans.cluster_centers_)
mapp = pd.DataFrame(inverse, columns=['longitude','latitude']).round(2)
print(mapp.shape)
maps = px.scatter_map(mapp,lat="latitude",
    lon="longitude",
    size='latitude',         
    zoom=11,
    center={"lat": 48.8566, "lon": 2.3522},
    map_style="carto-positron"  
    )
maps.show()



longitude    float64
latitude     float64
dtype: object
(7, 2)


In [9]:
wcss =  []
k = []
for i in range (1,11):
    kmeans2 = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans2.fit(X)
    wcss.append(kmeans2.inertia_)
    k.append(i)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))


WCSS for K=1 --> 275833.9999999999
WCSS for K=2 --> 155260.54651688985
WCSS for K=3 --> 89841.18709384027
WCSS for K=4 --> 66527.78905304271
WCSS for K=5 --> 50073.57551206917
WCSS for K=6 --> 41811.63874240588
WCSS for K=7 --> 35189.47614614988
WCSS for K=8 --> 31100.141458637587
WCSS for K=9 --> 27135.93464361711
WCSS for K=10 --> 24984.201219188464


On selectionne 5 clusters car moins de décroissance entre 6 et 7

Je teste maintenant la prédiction

In [10]:
second_pred = cluster_df.sample()
print(second_pred)
second_pred = sc.transform(second_pred)
results2 = kmeans.predict(second_pred)
print(f"L'appartement appartient au groupe : {results2[0]}")


       longitude  latitude
92820    2.39811   48.8664
L'appartement appartient au groupe : 6


Je veux voir la distribution des nouvelles catégories créée
pour cela je vais predict les zones des lignes 

In [11]:
df['zone'] = kmeans.predict(sc.transform(df[['longitude','latitude']]))

centers = pd.DataFrame(sc.inverse_transform(kmeans.cluster_centers_),columns=['longitude','latitude'])
centers["count"] = df['zone'].value_counts().sort_index()
centers['zone'] = df.zone.value_counts().sort_index().index
print(centers)

df['zone'].value_counts()


   longitude   latitude  count  zone
0   2.289726  48.845424  24860     0
1   2.356134  48.863229  18933     1
2   2.297028  48.876159  11698     2
3   2.392231  48.849304  17880     3
4   2.340956  48.833306  18866     4
5   2.338021  48.887694  27093     5
6   2.383429  48.876852  18587     6


zone
5    27093
0    24860
1    18933
4    18866
6    18587
3    17880
2    11698
Name: count, dtype: int64

In [12]:
new_maps = px.scatter_map(centers,lat="latitude",
    lon="longitude",
    size='count',         
    hover_name="zone",
    color='zone',
    hover_data={"count": True},
    zoom=11,
    center={"lat": 48.8566, "lon": 2.3522},
    map_style="carto-positron"  
    )
new_maps.show()

In [13]:
df[['zone','arrondissement']].head()


,zone,arrondissement
0,0,16
1,0,16
2,0,16
3,0,16
4,0,16
